In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import natsort
import _pickle
import cv2
import dlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path

from __utils__ import images_processing
from __utils__ import labels_processing
from __utils__ import labeling
from __utils__ import features_processing
from __utils__ import loso_preparing
from __utils__ import functions

In [ ]:
dataset_dir = "D:/Databases/CAS(ME)^2"
# dataset_dir = "F:/HEH/Databases/CAS(ME)^2"
# dataset_dir = "/data/disk1/heh/databases/CAS(ME)^2"

# dataset_dir = "D:/Databases/SAMM_longvideos"
# dataset_dir = "F:/HEH/Databases/SAMM_longvideos"
# dataset_dir = "/data/disk1/heh/databases/SAMM_longvideos"

test_dataset_dir = "D:/Databases/MEGC2022_testSet/CAS_Test_cropped"
# test_dataset_dir = "F:/HEH/Databases/MEGC2022_testSet/CAS_Test_cropped"
# test_dataset_dir = "/data/disk1/heh/databases/MEGC2022_testSet/CAS_Test_cropped"

# test_dataset_dir = "D:/Databases/MEGC2022_testSet/SAMM_Test_cropped"
# test_dataset_dir = "F:/HEH/Databases/MEGC2022_testSet/SAMM_Test_cropped"
# test_dataset_dir = "/data/disk1/heh/databases/MEGC2022_testSet/SAMM_Test_cropped"

images_loading = False
image_size = 128
load_cropped_images = False
# expression_type = "mae"  # macro-expression spotting
expression_type = "me"  # micro-expression spotting
save_x = False
debug_preds = True
labeling_function = "pseudo_labeling"
# labeling_function = "original_labeling"
model_names = {
    0: "SOFTNet",
    1: "SOFTNetCBAM",
    2: "ViT-B",
    3: "SL-ViT-B",
    4: "Swin-T",
    5: "Swin-S",
    6: "L-Swin-T",
    7: "S-Swin-T",
    8: "SL-Swin-T",
    9: "SL-Swin-S",
}
batch_size = 48
epochs = 25
save_preds = False

mae_model_name = model_names[8]
mae_preds_stem = (
    f"mae_"
    + mae_model_name.lower().replace("-", "_")
    + f"_batch_size_{batch_size}"
    + f"_epochs_{epochs}"
    + f"_{labeling_function}"
    + f"_{image_size}"
    # + "_1"
)
mae_preds_path = Path(dataset_dir, "preds", mae_preds_stem).with_suffix(".pkl")
print(f"mae_preds_path: {mae_preds_path}")

me_model_name = model_names[8]
me_preds_stem = (
    f"me_"
    + me_model_name.lower().replace("-", "_")
    + f"_batch_size_{batch_size}"
    + f"_epochs_{epochs}"
    + f"_{labeling_function}"
    + f"_{image_size}"
    # + "_3"
)
me_preds_path = Path(dataset_dir, "preds", me_preds_stem).with_suffix(".pkl")
print(f"me_preds_path: {me_preds_path}")

## Load Images 


When debug the image processing, the videos_images is from cropped_rawpic, whereas the other variables are from rawpic.


In [5]:
videos_images, subjects, subjects_videos_code = images_processing.load_images(
    dataset_dir, images_loading=images_loading, image_size=image_size
)

subject:  s15
subject:  s16
subject:  s19
subject:  s20
subject:  s21
subject:  s22
subject:  s23
subject:  s24
subject:  s25
subject:  s26
subject:  s27
subject:  s29
subject:  s30
subject:  s31
subject:  s32
subject:  s33
subject:  s34
subject:  s35
subject:  s36
subject:  s37
subject:  s38
subject:  s40


In [6]:
print("subjects:", subjects)
print("subjects_videos_code:", subjects_videos_code)

subjects: ['s15', 's16', 's19', 's20', 's21', 's22', 's23', 's24', 's25', 's26', 's27', 's29', 's30', 's31', 's32', 's33', 's34', 's35', 's36', 's37', 's38', 's40']
subjects_videos_code: [['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0508'], ['0101', '0102', '0401', '0402', '0502', '0505', '0507'], ['0102', '0402', '0505', '0507', '0502'], ['0502'], ['0101', '0401'], ['0101', '0102', '0402', '0503', '0508'], ['0102', '0402', '0503', '0507'], ['0101', '0401', '0402', '0502', '0507'], ['0101', '0102', '0502', '0508'], ['0101', '0102', '0401', '0503'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0502'], ['0101', '0102', '0401', '0502', '0503', '0505', '0507'], ['0101', '0401', '0402', '0502', '0503', '0505', '0507'], ['0101', '0102', '0401', '0402', '0502', '0503', '0505', '0507', '0508'], ['0102', '0402'], ['0401', '0402', '0503'], ['0102'], ['0401', '0505'], ['0101', '0402', '0502', '0505', '0507', '0508'], ['0502', '0507'], ['0401', '0502', '

## load Excel


In [8]:
Excel_data = labels_processing.load_excel(dataset_dir)
Excel_data.head(5)

,participant,video_name_&_expression_number,onset,apex,offset,AUs,extimated_emotion,expression_type,self-reported_emotion,video_name,video_code,subject
0,1,anger1_1,557,572,608,4+10+14+15,negative,macro-expression,anger,anger1,0401,s15
1,1,anger1_2,2854,2862,2871,38,others,macro-expression,sadness,anger1,0401,s15
2,1,anger2_1,2155,2163,0,NaN,negative,macro-expression,anger,anger2,0402,s15
3,1,anger2_2,3363,3371,3383,4+7+14,negative,macro-expression,anger,anger2,0402,s15
4,1,anger2_3,3380,3386,3407,4+14+38,negative,macro-expression,anger,anger2,0402,s15


## Load Ground Truth Labels


In [10]:
(
    mae_clean_videos_images,
    mae_clean_subjects_videos_code,
    mae_clean_subjects,
    mae_clean_subjects_videos_ground_truth_labels,
) = labels_processing.load_ground_truth_labels(
    dataset_dir,
    "mae",
    videos_images,
    subjects_videos_code,
    subjects,
    Excel_data,
)

required_videos_index:  [1, 4, 8, 9, 12, 13, 14, 16, 28, 33, 36, 37, 38, 45, 46, 47, 49, 50, 52, 54, 55, 57, 62, 64, 67, 71, 73, 74, 77, 83, 87, 91, 93]
len(clean_videos_images) = 33


In [11]:
print("len(mae_clean_subjects): ", len(mae_clean_subjects))
print("mae_clean_subjects: ", mae_clean_subjects)
print("len(mae_clean_subjects_videos_code): ", len(mae_clean_subjects_videos_code))
print("mae_clean_subjects_videos_codes: ", mae_clean_subjects_videos_code)
print(
    "len(mae_clean_subjects_videos_ground_truth_labels): ",
    len(mae_clean_subjects_videos_ground_truth_labels),
)
print(
    "mae_clean_subjects_videos_ground_truth_labels: ",
    mae_clean_subjects_videos_ground_truth_labels,
)

len(clean_subjects):  14
clean_subjects:  ['s15' 's16' 's19' 's23' 's24' 's25' 's27' 's29' 's30' 's31' 's32' 's35'
 's37' 's38']
len(clean_subjects_videos_code):  14
clean_subjects_videos_codes:  [['0102', '0502'], ['0101', '0102', '0502', '0505', '0507'], ['0402'], ['0102'], ['0401', '0507'], ['0101', '0102'], ['0101', '0102', '0401', '0502', '0503', '0507'], ['0502'], ['0101', '0401'], ['0101', '0402', '0505'], ['0401', '0502', '0503', '0508'], ['0102'], ['0402', '0508'], ['0507']]
len(clean_subjects_videos_ground_truth_labels):  14
clean_subjects_videos_ground_truth_labels:  [[[[698, 706]], [[137, 147]]], [[[551, 564]], [[269, 277]], [[322, 333]], [[395, 406], [1694, 1709], [1879, 1894]], [[1957, 1967], [2284, 2294]]], [[[1926, 1941]]], [[[330, 345], [525, 539], [726, 739]]], [[[607, 620], [962, 976], [1889, 1901], [2180, 2192], [3440, 3452]], [[1835, 1847], [1950, 1964], [3232, 3247]]], [[[112, 126]], [[995, 1007], [1007, 1016], [1017, 1033]]], [[[873, 887]], [[33, 47], [308, 316],

In [ ]:
(
    me_clean_videos_images,
    me_clean_subjects_videos_code,
    me_clean_subjects,
    me_clean_subjects_videos_ground_truth_labels,
) = labels_processing.load_ground_truth_labels(
    dataset_dir,
    "me",
    videos_images,
    subjects_videos_code,
    subjects,
    Excel_data,
)

In [ ]:
print("len(me_clean_subjects): ", len(me_clean_subjects))
print("me_clean_subjects: ", me_clean_subjects)
print("len(me_clean_subjects_videos_code): ", len(me_clean_subjects_videos_code))
print("me_clean_subjects_videos_codes: ", me_clean_subjects_videos_code)
print(
    "len(me_clean_subjects_videos_ground_truth_labels): ",
    len(me_clean_subjects_videos_ground_truth_labels),
)
print(
    "me_clean_subjects_videos_ground_truth_labels: ",
    me_clean_subjects_videos_ground_truth_labels,
)

## Calculate `k`


In [12]:
k = labels_processing.calculate_k(clean_subjects_videos_ground_truth_labels)

k (Half of average length of expression) =  6


In [ ]:
mae_k = labels_processing.calculate_k(mae_clean_subjects_videos_ground_truth_labels)
me_k = labels_processing.calculate_k(me_clean_subjects_videos_ground_truth_labels)

## Pseudo Labeling


In [ ]:
a = np.array([2, 3, 4])
b = 3
c = np.intersect1d(a, b)
print(len(c))

## Prepare for LOSO


## Debug Evaluation


In [3]:
import sys

epsilon = sys.float_info.epsilon

scenarios = {
    0: {
        "dataset": "CAS(ME)^2",
        "expression_type": "me",
        "num_expression": 57,
    },
    1: {
        "dataset": "CAS(ME)^2",
        "expression_type": "mae",
        "num_expression": 298,
    },
}

scenario = scenarios[1]
true_positive = 70
false_positive = 268
false_negative = scenario["num_expression"] - true_positive
print(
    f"True Positive: {true_positive} False Posive: {false_positive} False Negative: {false_negative}"
)
# add epsilon to avoid float division by zero
precision = true_positive / (true_positive + false_positive) + epsilon
recall = true_positive / (true_positive + false_negative) + epsilon
F1_score = (2 * precision * recall) / (precision + recall) + epsilon
print(f"Precision = {precision}, Recall ={recall}, F1-Score = {F1_score}")

True Positive: 70 False Posive: 268 False Negative: 228
Precision = 0.20710059171597656, Recall =0.23489932885906062, F1-Score = 0.22012578616352244


## Ablation Study


### MaE


In [ ]:
with open(mae_preds_path, "rb") as pkl_file:
    mae_preds = _pickle.load(pkl_file)
    pkl_file.close()

### ME


In [ ]:
with open(me_preds_path, "rb") as pkl_file:
    me_preds = _pickle.load(pkl_file)
    pkl_file.close()

In [20]:
"""
The 100-threads method is almost as fast as the 10-threads method.
"""

mae_ablation_dict = {
    "p": [],
    "true_positive": [],
    "false_positive": [],
    "false_negative": [],
    "expression_count": [],
    "precision": [],
    "recall": [],
    "F1_score": [],
}
me_ablation_dict = {
    "p": [],
    "true_positive": [],
    "false_positive": [],
    "false_negative": [],
    "expression_count": [],
    "precision": [],
    "recall": [],
    "F1_score": [],
}

print(" p | TP | FP | FN | Precision | Recall | F1-Score")
p_arange = np.arange(0.01, 1.0, 0.01)
mae_dict_queues = []
mae_threads = []
me_dict_queues = []
me_threads = []
for _ in p_arange:
    mae_dict_queues.append(Queue())
    me_dict_queues.append(Queue())

for p_index, p in enumerate(p_arange):
    # mae
    mae_thread = threading.Thread(
        target=functions.multithread_spot_and_evaluate,
        args=(
            mae_preds,
            mae_clean_subjects_videos_ground_truth_labels,
            mae_clean_videos_images,
            mae_clean_subjects,
            mae_clean_subjects_videos_code,
            mae_k,
            p,
            mae_dict_queues[p_index],
        ),
    )
    mae_thread.start()
    mae_threads.append(mae_thread)

    # me
    me_thread = threading.Thread(
        target=functions.multithread_spot_and_evaluate,
        args=(
            me_preds,
            me_clean_subjects_videos_ground_truth_labels,
            me_clean_videos_images,
            me_clean_subjects,
            me_clean_subjects_videos_code,
            me_k,
            p,
            me_dict_queues[p_index],
        ),
    )
    me_thread.start()
    me_threads.append(me_thread)
    for mae_thread, me_thread in zip(mae_threads, me_threads):
        mae_thread.join()
        me_thread.join()
    print(f" p: {p}", end="\r")

# Attention!!!
# Different loop sequence
for p_index, p in enumerate(p_arange):
    # mae
    mae_p_segment_matrix = mae_dict_queues[p_index].get()
    mae_ablation_dict["p"] += mae_p_segment_matrix["p"]
    mae_ablation_dict["true_positive"] += mae_p_segment_matrix["true_positive"]
    mae_ablation_dict["false_positive"] += mae_p_segment_matrix["false_positive"]
    mae_ablation_dict["false_negative"] += mae_p_segment_matrix["false_negative"]
    mae_ablation_dict["expression_count"] += mae_p_segment_matrix["expression_count"]
    mae_ablation_dict["precision"] += mae_p_segment_matrix["precision"]
    mae_ablation_dict["recall"] += mae_p_segment_matrix["recall"]
    mae_ablation_dict["F1_score"] += mae_p_segment_matrix["F1_score"]

    # me
    me_p_segment_matrix = me_dict_queues[p_index].get()
    me_ablation_dict["p"] += me_p_segment_matrix["p"]
    me_ablation_dict["true_positive"] += me_p_segment_matrix["true_positive"]
    me_ablation_dict["false_positive"] += me_p_segment_matrix["false_positive"]
    me_ablation_dict["false_negative"] += me_p_segment_matrix["false_negative"]
    me_ablation_dict["expression_count"] += me_p_segment_matrix["expression_count"]
    me_ablation_dict["precision"] += me_p_segment_matrix["precision"]
    me_ablation_dict["recall"] += me_p_segment_matrix["recall"]
    me_ablation_dict["F1_score"] += me_p_segment_matrix["F1_score"]

    # overall
    true_positive = (
        mae_ablation_dict["true_positive"][-1] + me_ablation_dict["true_positive"][-1]
    )
    false_positive = (
        mae_ablation_dict["false_positive"][-1] + me_ablation_dict["false_positive"][-1]
    )
    false_negative = (
        mae_ablation_dict["expression_count"][-1]
        + me_ablation_dict["expression_count"][-1]
        - true_positive
    )
    precision = true_positive / (true_positive + false_positive) + epsilon
    recall = true_positive / (true_positive + false_negative) + epsilon
    F1_score = (2 * precision * recall) / (precision + recall) + epsilon
    print(
        "{:.2f} | {} | {} | {} | {:.4f} | {:.4f} | {:.4f} |".format(
            p,
            true_positive,
            false_positive,
            false_negative,
            precision,
            recall,
            F1_score,
        )
    )

Split 0 is in process.
0 video(s) have been processed.
The current video be processed: subject s15, video 0102
The current video be processed: subject s15, video 0502

True Positive: 0 False Posive: 9 False Negative: 2
Precision = 2.220446049250313e-16, Recall =2.220446049250313e-16, F1-Score = 4.440892098500626e-16
Split 0 is processed.

Split 1 is in process.
2 video(s) have been processed.
The current video be processed: subject s16, video 0101
The current video be processed: subject s16, video 0102
The current video be processed: subject s16, video 0502
The current video be processed: subject s16, video 0505
The current video be processed: subject s16, video 0507

True Positive: 1 False Posive: 42 False Negative: 9
Precision = 0.023255813953488594, Recall =0.10000000000000023, F1-Score = 0.0377358490566043
Split 1 is processed.

Split 2 is in process.
7 video(s) have been processed.
The current video be processed: subject s19, video 0402

True Positive: 2 False Posive: 52 False Neg